In [1]:
import sqlite3

In [2]:
# connect to the database 
con = sqlite3.connect('./simulations.db')
# create a cursor object
cur = con.cursor()

In [3]:
# show content of database
cur.execute("SELECT name FROM sqlite_master")
print(cur.fetchall())

[('sqlite_sequence',), ('simulations',), ('parameters',)]


In [4]:
# to link tables
cur.execute("PRAGMA foreign_keys = ON;")
con.commit()
cur.execute("PRAGMA foreign_keys;")
print(cur.fetchall())

[(1,)]


In [5]:
# create table that does not allow duplicates
cur.execute("DROP TABLE simulations")
cur.execute("""
CREATE TABLE IF NOT EXISTS simulations (
            simulation_id integer PRIMARY KEY,
            N int NOT NULL,
            K int NOT NULL,
            lam float NOT NULL,
            mu float NOT NULL,
            h float NOT NULL,
            seed int NOT NULL,
            raw_file TEXT NOT NULL
            )
""")
cur.execute("CREATE UNIQUE INDEX parameters ON simulations (N,K,lam,mu,h);")
# can be dropped with cur.execute("DROP INDEX parameters;") if needs to be redesigned
con.commit()

In [6]:
def insert_simulation(simulation):
    # the sql command does not specify the values but only placeholders with ?
    sql=f"INSERT INTO simulations ({','.join(simulation.keys())}) VALUES ({','.join(['?']*len(simulation.keys()))})"
    # values are passed when executing the command
    cur.execute(sql, list(simulation.values()))
    con.commit()


# add an example simulation to the table 
simulation={'N':1000, 'K':100, 'lam':0.9, 'mu':0.2, 'h':0.01, 'seed':1000, 'raw_file':'path'}
insert_simulation(simulation)
# add another example simulation to the table
simulation['lam']=0.99
insert_simulation(simulation)

In [7]:
cur.execute("SELECT * FROM simulations")
print(cur.fetchall())

[(1, 1000, 100, 0.9, 0.2, 0.01, 1000, 'path'), (2, 1000, 100, 0.99, 0.2, 0.01, 1000, 'path')]


In [8]:
try:
    # this cannot be added because it is already in the database
    cur.execute("INSERT INTO simulations (N,K,lam,mu,h,seed,raw_file) VALUES (1000, 100, 0.99, 0.2, 0.01, 1000, 'path')")
except sqlite3.IntegrityError:
    print("This simulation is already in the database")
con.commit()
cur.execute("SELECT * FROM simulations")
print(cur.fetchall())

This simulation is already in the database
[(1, 1000, 100, 0.9, 0.2, 0.01, 1000, 'path'), (2, 1000, 100, 0.99, 0.2, 0.01, 1000, 'path')]


In [9]:
try:
    cur.execute("INSERT INTO simulations (N) VALUES (1000)")
except sqlite3.IntegrityError:
    print("IntegrityError because not all parameters are given")


IntegrityError because not all parameters are given


In [15]:
cur.execute("DROP TABLE IF EXISTS beta_approximations")
# add another table with beta approximation
cur.execute("""
CREATE TABLE IF NOT EXISTS beta_approximations (
            simulation_id INTEGER NOT NULL,
            a float NOT NULL,
            b float NOT NULL,
            loc float NOT NULL,
            scale float NOT NULL,
            FOREIGN KEY (simulation_id) REFERENCES simulations (simulation_id)
            )
""")
con.commit()
cur.execute("SELECT name FROM sqlite_master")
print(cur.fetchall())


[('sqlite_sequence',), ('simulations',), ('parameters',), ('beta_approximations',)]


In [19]:
# add beta approximation to the table
simulation={'N':1000, 'K':100, 'lam':0.99, 'mu':0.2, 'h':0.01, 'seed':1000, 'raw_file':'path'}
# get simulation ID from simulations table
cur.execute("SELECT simulation_id FROM simulations WHERE N=1000 AND K=100 AND lam=0.9 AND mu=0.2 AND h=0.01 AND seed=1000")
simulation_id=cur.fetchone()[0]
beta_approximation={'a':0.9, 'b':0.2, 'loc':0.01, 'scale':1000}
cur.execute(f"INSERT INTO beta_approximations (a,b,loc,scale, simulation_id) VALUES (0.9, 0.2, 0.01, 1000, {simulation_id})")
con.commit()
cur.execute("SELECT * FROM beta_approximations")
print(cur.fetchall())

[(1, 0.99, 0.2, 0.01, 1000.0), (1, 0.9, 0.2, 0.01, 1000.0)]


In [20]:
# update value in approximation
cur.execute("UPDATE beta_approximations SET a=0.99 WHERE simulation_id=1")
con.commit()
cur.execute("SELECT * FROM beta_approximations")
print(cur.fetchall())

[(1, 0.99, 0.2, 0.01, 1000.0), (1, 0.99, 0.2, 0.01, 1000.0)]
